## Data Parsing with sql database

# SQL DATABASE

In [ ]:
import sqlite3
import os


In [3]:
os.makedirs("data/databases", exist_ok=True)

In [4]:
## create sample databases
conn = sqlite3.connect('data/databases/company.db')
cursor=conn.cursor()

In [8]:
# Create employees table
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    role TEXT,
    department TEXT,
    salary REAL
)
""")

# Insert 5 employees
employees = [
    (1, "Alice Johnson", "Software Engineer", "IT", 75000),
    (2, "Bob Smith", "Data Analyst", "Data", 68000),
    (3, "Carol White", "Project Manager", "Operations", 82000),
    (4, "David Brown", "DevOps Engineer", "IT", 78000),
    (5, "Eva Green", "HR Manager", "HR", 70000)
]

cursor.executemany(
    "INSERT OR REPLACE INTO employees VALUES (?, ?, ?, ?, ?)",
    employees
)

In [9]:
# Create projects table
cursor.execute("""
CREATE TABLE IF NOT EXISTS projects (
    id INTEGER PRIMARY KEY,
    name TEXT,
    status TEXT,
    budget REAL,
    lead_id INTEGER
)
""")

# Insert 5 projects
projects = [
    (1, "RAG System", "Active", 120000, 1),
    (2, "Data Warehouse", "Completed", 90000, 2),
    (3, "Mobile App", "Active", 150000, 3),
    (4, "Cloud Migration", "Planning", 200000, 4),
    (5, "HR Portal", "Completed", 50000, 5)
]

cursor.executemany(
    "INSERT OR REPLACE INTO projects VALUES (?, ?, ?, ?, ?)",
    projects
)

In [ ]:
cursor.execute("SELECT * FROM employees")

In [ ]:
cursor.execute("SELECT * FROM projects")

In [18]:
conn.commit()
conn.close()

# Database Handling process

In [24]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader
from typing import List, Any
from langchain_core.documents import Document

In [23]:
## Method 1 - SQLDATABASE utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

# get db infor
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL:")
print(f"{db.get_table_info()}")

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	Alice Johnson	Software Engineer	IT	75000.0
2	Bob Smith	Data Analyst	Data	68000.0
3	Carol White	Project Manager	Operations	82000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG System	Active	120000.0	1
2	Data Warehouse	Completed	90000.0	2
3	Mobile App	Active	150000.0	3
*/


In [29]:
# Methid 2 - custom SQL to Document conversion
def sql_to_document(db_path:str) -> List:
    conn = sqlite3.connect(db_path)
    cursor=conn.cursor()
    documents=[]

    cursor.execute("SELECT name FROM sqlite_master WHERE Type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        
        # Get table schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]
        
        # Get table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        
        # Create table overview document
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"
        
        # Add sample records
        table_content += "Sample Records:\n"
        for row in rows[:5]:  # First 5 records
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
        
        doc = Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )
        documents.append(doc)

     # Strategy 2: Create relationship documents
    # Example: Join employees and projects
    cursor.execute("""
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
    """)
    
    relationships = cursor.fetchall()
    rel_content = "Employee-Project Relationships:\n\n"
    for rel in relationships:
        rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} - Status: {rel[3]}\n"
    
    rel_doc = Document(
        page_content=rel_content,
        metadata={
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee_project_join'
        }
    )
    documents.append(rel_doc)
    
    conn.close()
    return documents

In [30]:
sql_to_document("data/databases/company.db")

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 5, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 5\n\nSample Records:\n{'id': 1, 'name': 'Alice Johnson', 'role': 'Software Engineer', 'department': 'IT', 'salary': 75000.0}\n{'id': 2, 'name': 'Bob Smith', 'role': 'Data Analyst', 'department': 'Data', 'salary': 68000.0}\n{'id': 3, 'name': 'Carol White', 'role': 'Project Manager', 'department': 'Operations', 'salary': 82000.0}\n{'id': 4, 'name': 'David Brown', 'role': 'DevOps Engineer', 'department': 'IT', 'salary': 78000.0}\n{'id': 5, 'name': 'Eva Green', 'role': 'HR Manager', 'department': 'HR', 'salary': 70000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 5, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 5\n\nSample Records:\n{'id': 